In [3]:
import pandas as pd
df = pd.read_csv("emoji_df.csv")

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
# nltk.download('punkt')
# nltk.download('stopwords')

# preprocess text
def preprocess(text):
    tokens = word_tokenize(text)
    
    # remove punctuation
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    return tokens

groups = df['group'].unique()
sub_groups = df['sub_group'].unique()
df = df[df['name'] != 'thread']

from transformers import pipeline
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

# get most likely group based on a word
def get_group(word, groups):
    hypothesis_template = 'This text is about {}.'
    prediction = classifier(word, groups, hypothesis_template=hypothesis_template, multi_label=True)
    return [prediction['labels'][0], prediction['scores'][0]]

def word_to_emoji(word):
    # first look for direct matches
    if word in df.loc[df['name'] == word, 'name'].values:
        return df.loc[df['name'] == word, 'emoji'].values[0]
    group = get_group(word, groups)
    possible_emojis = df[df['group'] == group[0]]['name'].unique()
    name = get_group(word,possible_emojis)
    return [df[df['name'] == name[0]].iloc[0]['emoji'], name[1]]

In [4]:
# convert input text to emojis
user_input = input("Please enter your input text: ")
input_str = str(user_input)
print('Input text:', input_str)

input_str = preprocess(input_str)

emoji_translation = ''
for word in input_str:
    emoji = word_to_emoji(word)
    emoji_translation = emoji_translation + emoji[0]

print('emoji translation:',emoji_translation)

Input text: the house is on fire
emoji translation: 🏠🔥
